In [62]:
import rasterio
import geopandas as gpd
from rasterio.mask import mask
import numpy as np
from rasterio import features
from shapely.geometry import mapping, shape


class Masker:
    def __init__(self, data_dir, input_shp, input_raster, mask_raster):
        self.input_raster = rasterio.open(input_raster)
        self.input_shp = gpd.read_file(input_shp)
        self.mask_raster = rasterio.open(mask_raster)
        self.data_dir = data_dir
        
    def mask(self, filename="masked"):
        out_img, out_transform = mask(self.mask_raster, shapes=self.input_shp.geometry, crop=True)
        out_img[out_img == 0.0] = 255
        is_valid = (out_img != 255.0).astype(np.uint8)
#         return is_valid
        cropland = []
        for coords, value in features.shapes(is_valid, transform=out_transform):
#             print(value)
#             if value != 0:
            geom = shape(coords)
            cropland.append({"geometry": geom, "value" : value})
                
        cropland = gpd.GeoDataFrame(cropland).set_crs("epsg:4326")
        cropland = cropland[cropland['value']==1]
#         return cropland
        out_img, out_transform = mask(self.input_raster, cropland.geometry, crop=True)
#         return out_img
        out_img[np.isnan(out_img)] = 0
        out_img = out_img[0:24]
        with rasterio.open(
            f'{self.data_dir}/interim/{filename}.tif',
            'w',
            driver='GTiff',
            height=out_img.shape[1],
            width=out_img.shape[2],
            count=out_img.shape[0],
            dtype='float32',
            crs=self.input_raster.crs,
            transform=out_transform,
        ) as dst:
            dst.write(out_img[0:24])


In [63]:
msk = Masker("../../data", "../../data/inputs/aoi.gpkg", "../../data/interim/temp.tif", "/data/tmp/arogya/data/inputs/updated_mask.tif")

In [64]:
crop = msk.mask()
# crop[crop['value']==1].plot()
# msk.mask()[msk.mask()['value'==1]].plot()